# Implement Gradient Descent For Neural Network (or Logistic Regression)



In [56]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
df = pd.read_csv('/content/insurance_data.csv')
df.head(5)

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size = 0.2, random_state=142)

In [59]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [60]:
X_train_scaled.head(5)

,age,affordibility
0,0.22,1
7,0.60,0
27,0.46,1
6,0.55,0
15,0.55,1


In [61]:
model = keras.Sequential([
  keras.layers.Dense(1, input_shape = (2,), activation = 'sigmoid',kernel_initializer = 'ones' ,bias_initializer = 'zeros')
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    X_train_scaled,
    y_train,
    epochs = 5000
)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 14ms/step - loss: 0.5112 - accuracy: 0.7273
Epoch 2502/5000
1/1 [==============================] - 0s 14ms/step - loss: 0.5111 - accuracy: 0.7273
Epoch 2503/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5111 - accuracy: 0.7273
Epoch 2504/5000
1/1 [==============================] - 0s 15ms/step - loss: 0.5111 - accuracy: 0.7273
Epoch 2505/5000
1/1 [==============================] - 0s 14ms/step - loss: 0.5110 - accuracy: 0.7273
Epoch 2506/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5110 - accuracy: 0.7273
Epoch 2507/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5110 - accuracy: 0.7273
Epoch 2508/5000
1/1 [==============================] - 0s 18ms/step - loss: 0.5109 - accuracy: 0.7273
Epoch 2509/5000
1/1 [==============================] - 0s 14ms/step - loss: 0.5109 - accuracy: 0.7273
Epoch 2510/5000
1/1 [==========================

In [79]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 388ms/step - loss: 0.3871 - accuracy: 1.0000


[0.3871220052242279, 1.0]

In [63]:
X_test_pre = model.predict(X_test_scaled)
X_test_pre

1/1 [==============================] - 0s 87ms/step


array([[0.3910081 ],
       [0.3910081 ],
       [0.85683376],
       [0.66797435],
       [0.454252  ],
       [0.15398428]], dtype=float32)

In [64]:
X_test_pre[X_test_pre < 0.5] = 0
X_test_pre[X_test_pre >= 0.5] = 1

np.array(X_test_pre)

array([[0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.]], dtype=float32)

In [65]:
y_test = np.asarray(y_test)

In [66]:
y_test

array([0, 0, 1, 1, 0, 0])

In [67]:
coef, intercept = model.get_weights()

In [68]:
coef, intercept

(array([[5.191428 ],
        [1.6240107]], dtype=float32),
 array([-3.0015445], dtype=float32))

**This means w1 = 5.1914277, w2 = 1.6240108, bias = -3.0015445**

In [69]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

sigmoid(18)

0.9999999847700205

In [70]:
X_test

,age,affordibility
10,18,1
19,18,1
9,61,1
22,40,1
26,23,1
1,25,0


**Instead of model.predict, write our own prediction funtion that uses w1, w2 and bias**

In [71]:
def prediction_funsion(age, affordibility):
  weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
  return sigmoid(weighted_sum)

prediction_funsion(0.47,1)

<ipython-input-69-20547d567749>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))


0.7431560856518833

In [72]:
prediction_funsion(0.18,1)

<ipython-input-69-20547d567749>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))


0.39100811921166123

In [73]:
def sigmoid_numpy(x):
  return 1 / (1 + np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [74]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1 - epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return (-np.mean(y_true*np.log(y_predicted_new) + (1 - y_true)*np.log(1 - y_predicted_new)))

**All right now comes the time to implement our final gradient descent funsion**

In [75]:
X_train_scaled.head(5)

,age,affordibility
0,0.22,1
7,0.60,0
27,0.46,1
6,0.55,0
15,0.55,1


In [76]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age), (y_predicted - y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability), (y_predicted - y_true))

        bias_d = np.mean(y_predicted - y_true)
        w1 = w2 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print(f'Epochs: {i}, w1: {w1:.4f}, w2: {w2:.4f}, bias: {bias:.4f}, loss: {loss:.4f}')

        if loss<=loss_thresold:
            break

    return w1,w2,bias

In [77]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 5000, 0.4533)

Epochs: 0, w1: 0.9792, w2: 0.9695, bias: -0.0924, loss: 0.6562
Epochs: 1, w1: 0.9533, w2: 0.9457, bias: -0.1736, loss: 0.6377
Epochs: 2, w1: 0.9339, w2: 0.9282, bias: -0.2444, loss: 0.6236
Epochs: 3, w1: 0.9203, w2: 0.9164, bias: -0.3061, loss: 0.6134
Epochs: 4, w1: 0.9117, w2: 0.9093, bias: -0.3600, loss: 0.6058
Epochs: 5, w1: 0.9075, w2: 0.9064, bias: -0.4072, loss: 0.6002
Epochs: 6, w1: 0.9068, w2: 0.9067, bias: -0.4489, loss: 0.5960
Epochs: 7, w1: 0.9091, w2: 0.9098, bias: -0.4860, loss: 0.5927
Epochs: 8, w1: 0.9138, w2: 0.9151, bias: -0.5193, loss: 0.5901
Epochs: 9, w1: 0.9204, w2: 0.9221, bias: -0.5494, loss: 0.5879
Epochs: 10, w1: 0.9284, w2: 0.9305, bias: -0.5769, loss: 0.5860
Epochs: 11, w1: 0.9377, w2: 0.9400, bias: -0.6022, loss: 0.5843
Epochs: 12, w1: 0.9478, w2: 0.9502, bias: -0.6258, loss: 0.5827
Epochs: 13, w1: 0.9587, w2: 0.9611, bias: -0.6478, loss: 0.5812
Epochs: 14, w1: 0.9700, w2: 0.9724, bias: -0.6687, loss: 0.5799
Epochs: 15, w1: 0.9817, w2: 0.9839, bias: -0.6885,

(1.8901570213305636, 1.8760791826944596, -1.8110507065505856)

In [78]:
coef, intercept

(array([[5.191428 ],
        [1.6240107]], dtype=float32),
 array([-3.0015445], dtype=float32))